## Load Libs.

In [ ]:
!pip install langchain
!pip install openai

!pip install chromadb tiktoken
!pip install unstructured[local-inference] tabulate

after pip install step refress the session

In [ ]:
import os

import imaplib
import email
import getpass

from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
# define OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Ask a Question

In [ ]:
llm = OpenAI(temperature=0.01) # temprature: [0, 2]

text = "What is name of a company that jumping cat on its logo?"
print(llm(text))



The company is called Puma.


In [ ]:
llm = OpenAI(temperature=1.95) # temprature: [0, 2]

text = "What is name of a company that jumping cat on its logo?"
print(llm(text))



Nat ionalwTMaaWIltK Gro renpt.ines .ess


## Ask a Question Over a Text

In [ ]:
# IMAP server credentials
IMAP_SERVER = 'imap.gmail.com'
USERNAME = input("e-mail: ")
PASSWORD = getpass.getpass()

# Connect to the IMAP server
imap = imaplib.IMAP4_SSL(IMAP_SERVER)
imap.login(USERNAME, PASSWORD)

# Select the mailbox/folder containing the emails
mailbox = 'INBOX'
imap.select(mailbox)

# Open a text file to save the emails
output_file = open('all_emails.txt', 'w')

# Search for all emails
status, data = imap.search(None, 'ALL')
# Iterate over the email IDs and fetch the last 50 email data
for email_id in data[0].split()[-50:]:
    status, email_data = imap.fetch(email_id, '(RFC822)')
    raw_email = email_data[0][1]
    try:
        raw_email_string = raw_email.decode('utf-8')
    except UnicodeDecodeError:
        raw_email_string = raw_email.decode('latin-1')

    # Parse the raw email string into an email object
    email_message = email.message_from_string(raw_email_string)

    # Extract desired email information
    sender = email_message['From']
    subject = email_message['Subject']
    receiver = email_message["Delivered-To"]
    recieved_date = email_message['Received'].split('\r\n')[1]

    # Extract email body (plaintext)
    if email_message.is_multipart():
        for part in email_message.walk():
            if part.get_content_type() == 'text/plain':
                try:
                    body = part.get_payload(decode=True).decode('utf-8')
                except UnicodeDecodeError:
                    body = part.get_payload(decode=True).decode('latin-1')
                break
            else:
              body = "No text message!"
    else:
        try:
            body = email_message.get_payload(decode=True).decode('utf-8')
        except UnicodeDecodeError:
            body = email_message.get_payload(decode=True).decode('latin-1')

    # Process or save the extracted email information as needed
    output_file.write(f"Sender: {sender}\n")
    output_file.write(f"Receiver: {receiver}\n")
    output_file.write(f"Recieved_Date: {recieved_date}\n")
    output_file.write(f"Subject: {subject}\n")
    output_file.write(f"Body: {body}\n")
    output_file.write('\n\n')

# Close the output file
output_file.close()
# Logout from the IMAP server
imap.logout()

### Simple Model

In [ ]:
loader = TextLoader('/content/all_emails.txt')
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
# query your index
query = "Who is the last message from?"
index.query(query)

### Custom Model

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
def repeted_question(qa, question):
    chat_history = []
    while True:
        result = qa({"question": question,
                     "chat_history": chat_history})
        chat_history.append((question, result['answer']))
        print(f"**Answer**: {result['answer']} \n")

        another_question = input("Do you have another question? (If not say 'no'): ")
        if another_question.lower() != "no":
            question = another_question.copy()
            break
        else:
            print("The conversation is over")
    return

In [ ]:
# give path of document/documents
loader = DirectoryLoader("/content/", glob="*.txt")
txt_docs = loader.load_and_split()

# prepare the text
embeddings = OpenAIEmbeddings()
txt_docsearch = Chroma.from_documents(txt_docs, embeddings)

# define model and args.
llm = ChatOpenAI(model_name="gpt-3.5-turbo",
                 temperature=0.5)
qa = ConversationalRetrievalChain.from_llm(llm,
                                           retriever=txt_docsearch.as_retriever())

In [ ]:
# ask a question
repeted_question(qa,
                 "What is the content of the message from Ahold?")